In [ ]:
# LangChain V0.3版本
!pip install langchain==0.3.18 \
    langchain-community==0.3.17 \
    langchain-core==0.3.34 \
    langchain-openai==0.3.4 \
    chromadb==0.6.3

In [ ]:
# 导入相关的 LangChain 功能模块
# 从 langchain_openai 库导入 ChatOpenAI 和 OpenAIEmbeddings，用于使用 OpenAI 的聊天模型和嵌入模型
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults  # 导入 Tavily 搜索引擎工具
from langchain_core.messages import HumanMessage  # 导入 HumanMessage 类型，用于表示用户输入
from langgraph.checkpoint.memory import MemorySaver  # 导入 MemorySaver，用于内存存储对话历史
from langgraph.prebuilt import create_react_agent  # 导入 create_react_agent 函数，用于创建 React Agent

In [ ]:
# 创建 Agent 的组件

# 初始化内存存储器，用于保存对话历史 (可选，用于有状态 Agent)
memory = MemorySaver()

# Create a ChatOpenAI model
# 智谱API调用(glm-4v-flash) - 使用 ChatOpenAI 替换 ChatAnthropic，并配置智谱 API
model = ChatOpenAI(
    api_key="XXXX", # 替换为您的智谱 API 密钥  (请务必替换成您自己的 API 密钥)
    base_url="https://open.bigmodel.cn/api/paas/v4/", # 智谱 API Base URL
    model="glm-4v-flash" # 使用 glm-4v-flash 模型
)

# 初始化 Tavily 搜索引擎工具，设置最大搜索结果数为 2
search = TavilySearchResults(max_results=2)
# 将 Tavily 搜索引擎工具放入工具列表，Agent 可以使用这些工具
tools = [search]

# 使用 `create_react_agent` 函数创建 Agent 执行器
# 该函数会自动将模型和工具绑定，并使用 ReAct 框架构建 Agent
# checkpointer=memory 参数用于启用对话记忆功能
agent_executor = create_react_agent(model, tools, checkpointer=memory)


# ---  首次对话：打招呼并告知所在地 ---
# 配置对话线程 ID，用于记忆功能，相同 thread_id 共享对话历史
config = {"configurable": {"thread_id": "abc123"}}
print("--- 首次对话 ---")
# 使用 stream 方法进行流式输出，逐步显示 Agent 的响应
for chunk in agent_executor.stream(
    # 构造用户输入消息 HumanMessage -  用户用中文打招呼，并告知住在北京
    {"messages": [HumanMessage(content="你好！我是小明，我住在北京。")]}, config
):
    # 打印每个 chunk 的输出，展示 Agent 的思考过程
    print(chunk)
    print("----")


# ---  第二次对话：询问天气 ---
print("\n--- 第二次对话 ---")
for chunk in agent_executor.stream(
    # 再次使用相同的 thread_id，Agent 将记住之前的对话 -  用户用中文询问天气
    {"messages": [HumanMessage(content="我住的地方天气怎么样？")]}, config
):
    print(chunk)
    print("----")


# ---  第三次对话： 询问名字 ---
print("\n--- 第三次对话 (验证记忆) ---")
for chunk in agent_executor.stream(
    # 继续使用相同的 thread_id -  用户用中文询问名字
    {"messages": [HumanMessage(content="请问我叫什么名字？")]}, config
):
    print(chunk)
    print("----")


# ---  新的对话：更换线程 ID，验证无记忆 ---
# 使用新的 thread_id，开启新的对话
config_new_thread = {"configurable": {"thread_id": "xyz123"}}
print("\n--- 新的对话 (无记忆) ---")
for chunk in agent_executor.stream(
    # 用户在新对话线程中，用中文询问名字
    {"messages": [HumanMessage(content="我叫什么名字？")]}, config_new_thread
):
    print(chunk)
    print("----")


print("\n---  代码执行完成  ---")